При обработке 10 страниц - время работы составляет 10 минут

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Базовый URL для пагинации на сайте Lifehacker
BASE_URL = "https://lifehacker.ru/topics/technology/"

# Функция для получения HTML-кода страницы
def get_html(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Проверяем статус ответа
        return response.text
    except requests.RequestException as e:
        print(f"Ошибка при запросе {url}: {e}")
        return None

# Функция для парсинга списка статей
def parse_article_list(html):
    soup = BeautifulSoup(html, 'lxml')
    articles = []

    # Ищем все элементы статей
    for article in soup.find_all('a', class_='lh-small-article-card__link'):
        title = article.get('title', "Нет заголовка")  # Заголовок статьи из атрибута title
        link = article.get('href')  # Ссылка на статью

        # Преобразуем относительные ссылки в абсолютные
        if link.startswith('/'):
            link = f"https://lifehacker.ru{link}"

        articles.append({
            'title': title,
            'link': link
        })
    return articles

# Функция для парсинга содержимого статьи
def parse_article_content(html, title):
    soup = BeautifulSoup(html, 'lxml')

    # Ищем все параграфы текста
    paragraphs = soup.find_all('p')  # Находим все теги <p>

    # Пропускаем первый параграф и начинаем со второго, так как первый это про автора
    if len(paragraphs) > 1:
        paragraphs = paragraphs[1:]  # Пропускаем первый параграф

    # Объединяем текст из всех оставшихся параграфов
    if paragraphs:
        content_text = "\n".join(p.get_text(strip=True) for p in paragraphs)
        if not content_text:  # Если текст пустой
            content_text = "Текст статьи отсутствует"
    else:
        content_text = "Текст статьи отсутствует"

    return {
        'title': title,  # Используем заголовок, полученный из списка статей
        'content': content_text  # Полный текст статьи
    }

# Собираем ссылки со всех страниц
parsed_urls = []

for page_num in range(1, 11):  # Обрабатываем первые 10 страниц (ЗАНИМАЕТ 10 МИНУТ)
    url = BASE_URL if page_num == 1 else f"{BASE_URL}?page={page_num}"
    print(f"Обработка страницы {page_num}: {url}")

    html = get_html(url)
    if not html:
        continue

    articles = parse_article_list(html)
    if not articles:
        print(f"Статьи не найдены на странице {url}")
        break

    parsed_urls.extend(articles)

print(f"Найдено {len(parsed_urls)} ссылок на статьи.")

# Собираем данные по всем статьям
all_articles = []

for article in parsed_urls:
    print(f"Парсинг статьи: {article['title']}\n{article['link']}")

    article_html = get_html(article['link'])  # Получаем HTML-код статьи
    if article_html:
        article_data = parse_article_content(article_html, article['title'])  # Передаем заголовок
        all_articles.append(article_data)
    else:
        print(f"Не удалось получить данные для статьи: {article['title']}")

# Создаем DataFrame
df = pd.DataFrame(all_articles)

# Выводим первые строки DataFrame
print(df.head())

# Сохраняем в CSV
if not df.empty:
    df.to_csv("lifehacker_articles.csv", index=False, encoding='utf-8')
    print("Данные сохранены в файл lifehacker_articles.csv")
else:
    print("Нет данных для сохранения.")


Обработка страницы 1: https://lifehacker.ru/topics/technology/
Найдено 30 ссылок на статьи.
Парсинг статьи: Как очистить экран телевизора без разводов и вреда для матрицы
https://lifehacker.ru/kak-ochistit-ekran-televizora-bez-razvodov/
Парсинг статьи: Как выбрать хорошие беспроводные наушники: подробное руководство
https://lifehacker.ru/kak-vybrat-besprovodnye-naushniki/
Парсинг статьи: В поисковике Google появится ИИ-генератор подкастов на основе результатов выдачи
https://lifehacker.ru/generator-podskastov-v-google/
Парсинг статьи: 13 смартфонов с большими скидками на «Фестивале брендов» AliExpress
https://lifehacker.ru/festival-brendov-2025-smartfony/
Парсинг статьи: 10 лучших ТВ до 20 000 рублей, которые вас удивят
https://lifehacker.ru/luchshie-nedorogie-televizory/
Парсинг статьи: Garmin представила Venu X1 с 2‑дюймовым экраном — ответ на Apple Watch Ultra&nbsp;2
https://lifehacker.ru/garmin-venu-x1-anons/
Парсинг статьи: Трио часов Samsung Galaxy Watch 8, 8 Classic и Ultra 2025

In [15]:
# Открываем и выводим содержимое CSV-файла
df_loaded = pd.read_csv("lifehacker_articles.csv")
display(df_loaded.head())  # Выводим первые 5 строк для проверки

,title,content
0,Как очистить экран телевизора без разводов и в...,Мы привыкли вытирать экран смартфона чем попал...
1,Как выбрать хорошие беспроводные наушники: под...,"Покупка наушников — непростая задача, особенно..."
2,В поисковике Google появится ИИ-генератор подк...,Googleтестируетв своём поисковике функцию Audi...
3,13 смартфонов с большими скидками на «Фестивал...,С 16 по 26 июня на AliExpress пройдёт «Фестива...
4,"10 лучших ТВ до 20 000 рублей, которые вас удивят",Хороший телевизор до 20 тысяч рублей — это не ...
